# Day 4: Ceres Search

## Part 1

"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with `.:`

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```
The actual word search will be full of letters instead. For example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with `.:`

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

Take a look at the little Elf's word search. How many times does XMAS appear?

In [50]:
xmas = "XMAS"


def get_data(filename='data.txt'):
    with open(filename) as f:
        lines = f.read().splitlines()
        return [list(l) for l in lines]


data = get_data('data.txt')

### Strategy
We need to look for:

```
XMAS  SAMX

X  S  X     S        X
M  A   M     A      M
A  M    A     M    A
S  X     S     X  S

etc.
```

#### Naive solution
The naive solution would be to find every X in the input and simply check all (up to) 9 adjacent spaces. For instance, with the example:

```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```

We would stop at `(0, 4)` and look for an M at `[(0, 3), (0, 5), (1, 3), (1, 4), (1, 5)]`, which we would find at `[(1, 3), (1, 5)]`. We would then search for an A at each of those, and finally an S at any subsequent matches. 

One of the big issues with this solution is keeping track of the bounds. For instance, an X in row 3 (index 2) couldn't possibly spell "XMAS" because there are only 2 rows above it. 

In [59]:
debug = True

from enum import Enum
import time


class Direction(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4
    UP_LEFT = 5
    UP_RIGHT = 6
    DOWN_LEFT = 7
    DOWN_RIGHT = 8


def get_neighbors(row, col, n, m, direction: Direction = None, target=None):
    """This method is used to get all adjacent coordinates, respecting the bounds of the matrix"""
    assert -1 < row < n
    assert -1 < col < m
    all_neighbors = [
        (i, j) for i in [row - 1, row, row + 1] for j in [col - 1, col, col + 1] if (i, j) != (row, col)
    ]

    # Remove any out of bounds
    all_neighbors = list(filter(lambda x: 0 <= x[0] < n and 0 <= x[1] < m, all_neighbors))

    if target:
        all_neighbors = list(filter(lambda x: data[x[0]][x[1]] == target, all_neighbors))

    condition = None
    if direction == Direction.UP:
        condition = lambda x: x[0] < row and x[1] == col
    elif direction == Direction.DOWN:
        condition = lambda x: x[0] > row and x[1] == col
    elif direction == Direction.LEFT:
        condition = lambda x: x[0] == row and x[1] < col
    elif direction == Direction.RIGHT:
        condition = lambda x: x[0] == row and x[1] > col
    elif direction == Direction.UP_LEFT:
        condition = lambda x: x[0] < row and x[1] < col
    elif direction == Direction.UP_RIGHT:
        condition = lambda x: x[0] < row and x[1] > col
    elif direction == Direction.DOWN_LEFT:
        condition = lambda x: x[0] > row and x[1] < col
    elif direction == Direction.DOWN_RIGHT:
        condition = lambda x: x[0] > row and x[1] > col
    
    if condition:
        all_neighbors = list(filter(condition, all_neighbors))
    

    return all_neighbors


def get_direction(n1, m1, n2, m2):
    if n1 == n2:
        return Direction.RIGHT if m1 < m2 else Direction.LEFT
    elif m1 == m2:
        return Direction.DOWN if n1 < n2 else Direction.UP
    elif n1 < n2:
        return Direction.DOWN_RIGHT if m1 < m2 else Direction.DOWN_LEFT
    elif n1 > n2:
        return Direction.UP_RIGHT if m1 < m2 else Direction.UP_LEFT
    else:
        return None


def scan(grid, row, col, target, direction: Direction = None):
    next_target = "M" if target == "X" else "A" if target == "M" else "S" if target == "A" else None
    count, n, m = 0, len(data), len(data[0])
    actual = grid[row][col]

    # Base case
    if actual != target: return 0
    if target == actual == "S": return 1

    # Get neighbors constrained by direction and target
    neighbors = get_neighbors(row, col, n, m, direction, target=next_target)
    for neighbor in neighbors:
        # We only need to get the direction if we're at an X
        if target == "X":
            direction = get_direction(row, col, neighbor[0], neighbor[1])

        # If the neighbor is the next target, we can continue scanning
        if grid[neighbor[0]][neighbor[1]] == next_target:
            count += scan(grid, neighbor[0], neighbor[1], next_target, direction)

    return count


def count_xmas(data):
    total = 0
    n = len(data)
    m = len(data[0])
    for i in range(n):
        for j in range(m):
            if data[i][j] == "X":
                total += scan(data, i, j, "X")
    return total

print(count_xmas(data))

2464
